In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt # for 2D visulization
import seaborn as sns # for 2D visulization

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from scipy import stats # for statistics

from nltk.sentiment import SentimentIntensityAnalyzer # sentiment analysis
import nltk
import warnings #filtering our warnings

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
d_df = pd.read_csv('/content/chotRFClassifier_job+com+acc+no++cre.csv')
d_df

,LABOUR_CONTRACT_TYPE,NUMBER_OF_DEPENDANTS,PERMANENT_ADDRESS_PROVINCE,INDUSTRY,JOB,COMPANY_ADDRESS_PROVINCE,CONTRACT_NO,WEIGHT,EDUCATION,CUSTOMER_INCOME,CREDITIBILITY,ACCOMMODATION_TYPE,DATE_OF_BIRTH,HEIGHT,WORKING_IN_YEAR,MARITAL_STATUS,INCOME_RESOURCE
0,2.0,2.0,20.0,18.0,9.0,74.0,NaN,60.0,1.0,6056000.0,1.0,0.0,1979.0,155.0,1.0,2.0,5.0
1,4.0,0.0,58.0,18.0,6.0,58.0,569518.0,73.1,2.0,19092000.0,0.0,0.0,1969.0,178.0,2.0,1.0,1.0
2,4.0,0.0,NaN,2.0,9.0,74.0,569723.0,65.0,1.0,1936000.0,2.0,3.0,1964.0,189.0,16.0,4.0,5.0
3,4.0,1.0,46.0,18.0,4.0,46.0,NaN,83.0,2.0,15413000.0,0.0,0.0,NaN,168.0,3.0,2.0,5.0
4,4.0,3.0,NaN,18.0,9.0,79.0,571556.0,74.0,1.0,17533000.0,2.0,0.0,1974.0,172.0,8.0,2.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371195,4.0,3.0,75.0,18.0,1.0,75.0,630947.0,82.0,6.0,19262000.0,3.0,0.0,1976.0,156.0,2.0,2.0,NaN
371196,4.0,0.0,94.0,NaN,1.0,79.0,823371.0,99.0,NaN,17533000.0,4.0,4.0,NaN,175.0,NaN,0.0,5.0
371197,NaN,1.0,75.0,18.0,9.0,75.0,631768.0,63.0,3.0,12520000.0,0.0,0.0,1994.0,141.0,3.0,2.0,5.0
371198,4.0,0.0,45.0,18.0,6.0,45.0,NaN,97.0,NaN,17533000.0,3.0,0.0,1981.0,189.0,NaN,2.0,1.0


In [ ]:
d_df.isna().sum()

,0
LABOUR_CONTRACT_TYPE,78152
NUMBER_OF_DEPENDANTS,0
PERMANENT_ADDRESS_PROVINCE,55829
INDUSTRY,67106
JOB,0
COMPANY_ADDRESS_PROVINCE,0
CONTRACT_NO,55693
WEIGHT,0
EDUCATION,62964
CUSTOMER_INCOME,0


In [ ]:
train_data = d_df[d_df['PERMANENT_ADDRESS_PROVINCE'].notna()].copy()  # Dữ liệu không bị thiếu
test_data = d_df[d_df['PERMANENT_ADDRESS_PROVINCE'].isna()].copy()     # Dữ liệu bị thiếu

# Bước 2: Mã hóa các giá trị trong cột 'PERMANENT_ADDRESS_PROVINCE' từ chuỗi thành số
label_encoder = LabelEncoder()
train_data['PERMANENT_ADDRESS_PROVINCE_encoded'] = label_encoder.fit_transform(train_data['PERMANENT_ADDRESS_PROVINCE'])

# Bước 3: Chọn các đặc trưng cho huấn luyện và nhãn
# Chọn tất cả các cột đặc trưng (ngoại trừ 'PERMANENT_ADDRESS_PROVINCE' và 'PERMANENT_ADDRESS_PROVINCE_encoded')
X_train = train_data.drop(columns=['PERMANENT_ADDRESS_PROVINCE', 'PERMANENT_ADDRESS_PROVINCE_encoded'])
y_train = train_data['PERMANENT_ADDRESS_PROVINCE_encoded']

# Bước 4: Chuẩn bị các đặc trưng cho dữ liệu thiếu
X_test = test_data.drop(columns=['PERMANENT_ADDRESS_PROVINCE'])

# Bước 5: Khởi tạo và huấn luyện mô hình RandomForestClassifier
model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# Bước 6: Dự đoán các giá trị missing trong cột 'PERMANENT_ADDRESS_PROVINCE'
predicted_values_encoded = model.predict(X_test)

# Bước 7: Chuyển các giá trị dự đoán từ số trở lại thành chuỗi (sử dụng LabelEncoder)
predicted_values = label_encoder.inverse_transform(predicted_values_encoded)

# Bước 8: Điền giá trị dự đoán vào cột 'PERMANENT_ADDRESS_PROVINCE' trong DataFrame d_df
d_df.loc[d_df['PERMANENT_ADDRESS_PROVINCE'].isna(), 'PERMANENT_ADDRESS_PROVINCE'] = predicted_values
d_df.to_csv('/content/drive/MyDrive/chotRFClassifier_job+com+acc+no++cre+per.csv', index=False)


In [ ]:
d_df.isna().sum()

,0
LABOUR_CONTRACT_TYPE,78152
NUMBER_OF_DEPENDANTS,0
PERMANENT_ADDRESS_PROVINCE,0
INDUSTRY,67106
JOB,0
COMPANY_ADDRESS_PROVINCE,0
CONTRACT_NO,55693
WEIGHT,0
EDUCATION,62964
CUSTOMER_INCOME,0
